In [32]:
# import dependencies
import pandas as pd
import requests
import numpy as np
import matplotlib
from keys import api_key
import json
from datetime import datetime # dt_object = datetime.fromtimestamp(timestamp)

In [34]:
# bring in stock List and clean it up
start_year = 1989
end_year = 2020
year_list = list(np.arange(start_year, end_year, 1))

f500_df_prelim = pd.DataFrame(columns=['rank', 'company', 'year'])

for year in year_list:
    
    path = f'Resources/csv/fortune500-{year}.csv'
    
    f500_df = pd.read_csv(path)
    f500_df['year'] = year
    
    f500_df_prelim = f500_df_prelim.append(f500_df)
    
    f500_df_prelim.reset_index()
    
    
f500_df = f500_df_prelim

listoflists = pd.DataFrame(f500_df['company'].value_counts()).reset_index().rename(columns={'index':'company', 'company':'occurance'})
f500_df_sort = listoflists.merge(f500_df, on='company', how='left')
f500_df_sort = f500_df_sort.loc[f500_df_sort['profit ($ millions)'] != 'N.A.']
f500_df_sort['profit ($ millions)'] = f500_df_sort['profit ($ millions)'].astype('float64')
df2 = f500_df_sort.groupby('company').max().sort_values('profit ($ millions)', ascending=False)
df2 = df2.loc[df2['occurance'] >= 25]
df2 = df2.reset_index()
df2.head(2)

,company,occurance,profit ($ millions),rank,revenue ($ millions),year
0,Exxon Mobil,31,45220.0,4,452926.0,2019
1,Berkshire Hathaway,31,44940.0,295,247837.0,2019


In [35]:
df3 = pd.read_csv('stock_ticker_info.csv',  encoding="ISO-8859-1")
df3 = df3.rename(columns={'Company Name2': 'company'})
df3.head(2)

,company,Name,Ticker
0,Dean Foods,Dean Foods Company,DF
1,OfficeMax,"Office Depot, Inc.",ODP


In [38]:
df2 = df2.merge(df3,on='company', how='left')

In [40]:
df2 = df2[['Name', 'Ticker']]
df2.head(2)

,Name,Ticker
0,Exxon Mobil Corporation,XOM
1,Berkshire Hathaway Inc.,BRK-A


In [41]:
df2.to_csv('Fortune_list.csv')

In [57]:
# create list from df
symbols = df2['Ticker'].tolist()

response_df = pd.DataFrame(columns=['Symbol', 'Date', 'Close', 'Volume'])

# containters
# date = []
# close = []
# volume = []

# loop through df and run search for each stock symbol
for symbol in symbols:
    base_url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-historical-data"
    
    headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': api_key
    }
    
    params = {"frequency":"1wk","filter":"history","period1":"599637600","period2":"1577858399","symbol":symbol}
    
    responses = requests.get(base_url, headers=headers, params=params).json()
    
    for response in responses:
        response_df['Symbol'] = response_df.append(symbol)
        response_df['Date'] = response_df.append(response["prices"][X]["date"])
        response_df['Close'] = response_df.append(response["prices"][X]["close"])
        response_df['Volume'] = response_df.append(response["prices"][X]["volume"])
        
response_df.head()

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

,Symbol,Date,Close,Volume
